In [1]:
from typing import Union
from pathlib import Path
from copy import deepcopy
import json
from datetime import datetime

import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader

from s4hci.models.noise import S4Noise
from s4hci.models.planet import S4Planet
from s4hci.models.normalization import S4FrameNormalization
from s4hci.utils.adi_tools import combine_residual_stack
from s4hci.utils.data_handling import save_as_fits, load_adi_data
from s4hci.models.rotation import FieldRotationModel

from applefy.utils.file_handling import open_fits
from applefy.utils.fake_planets import add_fake_planets

In [2]:
torch.set_float32_matmul_precision("high")

# Load the data and insert the fake planet

In [3]:
# 2.) Load the dataset
science_data, raw_angles, raw_psf_template_data = \
    load_adi_data(
        hdf5_dataset="/fast/mbonse/s4/30_data/HD22049_303_199_C-0065_C_.hdf5",
        data_tag="object",
        psf_template_tag="psf_template",
        para_tag="header_object/PARANG")

science_data = science_data[::2, 12:-12, 12:-12]
raw_angles = raw_angles[::2]

# Background subtraction of the PSF template
psf_template_data = np.median(raw_psf_template_data, axis=0)
psf_template_data = psf_template_data - np.min(psf_template_data)

# Implement the current training loop

In [4]:
class RotationS4:
    
    def __init__(
        self,
        science_data,
        psf_template,
        lambda_reg,
        parang,
        inverse=1,
        conv=False,
        work_dir=None,
        psf_cut_radius=4.0,
        mask_radius=5.5):

        # 1.) Save the data
        self.device = 0
        self.science_data = torch.from_numpy(science_data).float()
        self.psf_template = psf_template
        self.data_image_size = self.science_data.shape[-1]
        self.parang = parang
        
        if work_dir is not None:
            self.work_dir = Path(work_dir)
        else:
            self.work_dir = None
        self.residuals_dir, self.tensorboard_dir, self.models_dir = \
            self._setup_working_dir()

        # 2.) Create the noise model
        self.noise_model = S4Noise(
            data_image_size=self.data_image_size,
            psf_template=psf_template,
            lambda_reg=lambda_reg,
            cut_radius_psf=psf_cut_radius,
            mask_template_setup=("radius", mask_radius),
            convolve=conv,
            verbose=True).float()
        
        # 2.) Create the filed rotation model
        self.rotation_model = FieldRotationModel(
            all_angles=parang * inverse, # TODO check if this is the right direction
            input_size=self.data_image_size,
            subsample=1,
            inverse=False,
            register_grid=True)

        # 3.) Create normalization model
        self.normalization_model = S4FrameNormalization(
            image_size=self.data_image_size,
            normalization_type="normal")

        self.normalization_model.prepare_normalization(
            science_data=self.science_data)
        
        # 4.) For tensorboard
        self.tensorboard_logger = None
        self.fine_tune_start_time = None
        
    def _setup_working_dir(self):
        if self.work_dir is None:
            return None, None, None

        # make sure the working dir is a dir
        self.work_dir.mkdir(exist_ok=True)

        residuals_dir = self.work_dir / "residuals"
        tensorboard_dir = self.work_dir / "tensorboard"
        models_dir = self.work_dir / "models"

        residuals_dir.mkdir(exist_ok=True)
        tensorboard_dir.mkdir(exist_ok=True)
        models_dir.mkdir(exist_ok=True)

        return residuals_dir, tensorboard_dir, models_dir
        
    def _create_tensorboard_logger(self):
        time_str = datetime.now().strftime("%Y-%m-%d-%Hh%Mm%Ss")
        self.fine_tune_start_time = time_str
        current_logdir = self.tensorboard_dir / \
            Path(self.fine_tune_start_time)
        current_logdir.mkdir()
        self.tensorboard_logger = SummaryWriter(current_logdir)
        
    def _logg_loss_values(
            self,
            epoch,
            loss_recon,
            loss_reg):

        if self.work_dir is None:
            return

        self.tensorboard_logger.add_scalar(
            "Loss/Reconstruction_loss",
            loss_recon,
            epoch)

        self.tensorboard_logger.add_scalar(
            "Loss/Regularization_loss",
            loss_reg,
            epoch)
        
    
    @staticmethod
    def _normalize_for_tensorboard(frame_in):
        image_for_tb = deepcopy(frame_in)
        image_for_tb -= np.min(image_for_tb)
        image_for_tb /= np.max(image_for_tb)
        return image_for_tb
    
    @torch.no_grad()
    def compute_residual(
            self,
            num_cpus=8,
            subtract_temporal_average=False
    ):
        # 1.) normalize and reshape the data
        x_norm = self.normalization_model(self.science_data)
        science_norm_flatten = x_norm.view(x_norm.shape[0], -1)

        # 2.) compute the noise estimate
        noise_estimate = self.noise_model(science_norm_flatten)

        # 3.) compute and reshape the residual sequence
        residual_sequence = science_norm_flatten - noise_estimate
        residual_stack = residual_sequence.view(
            self.science_data.shape[0],
            self.noise_model.image_size,
            self.noise_model.image_size).detach().cpu().numpy()

        # 4.) derotate and stack the residual sequence
        residual_image = combine_residual_stack(
            residual_stack=residual_stack,
            angles=self.parang,
            combine="mean",
            subtract_temporal_average=subtract_temporal_average,
            num_cpus=num_cpus)

        return residual_image

# Implement LBFGS

In [5]:
dataset_id = "0149b"

# add the fake planet
fake_planet_config_file = "/fast/mbonse/s4/70_results/07_partial_contrast_grid/HD22049_303_199_C-0065_C_/configs_cgrid/exp_ID_" + dataset_id + ".json"
with open(fake_planet_config_file) as json_file:
    fake_planet_config = json.load(json_file)

data_with_fake_planet = add_fake_planets(
    input_stack=science_data,
    psf_template=psf_template_data,
    parang=raw_angles,
    dit_psf_template=0.004256,
    dit_science=0.08,
    experiment_config=fake_planet_config,
    scaling_factor=1.0)

In [ ]:
device = 0
batch_size=-1
num_epochs=500
lambda_reg=0

# Create the model
s4_model = RotationS4(
    science_data=data_with_fake_planet,
    psf_template=psf_template_data,
    lambda_reg=lambda_reg,
    inverse=1,
    conv=True,
    mask_radius=5.5,
    work_dir="/fast/mbonse/s4/70_results/09_new_rotation_loss/fft_loss/",
    parang=raw_angles)

# Create the tensorboard logger
if s4_model.work_dir is not None:
    s4_model._create_tensorboard_logger()

# 1.) normalize the science data
x_norm = s4_model.normalization_model(s4_model.science_data)
science_norm_flatten = x_norm.view(x_norm.shape[0], -1)

# 2.) move models to the GPU
s4_model.noise_model = s4_model.noise_model.to(device)
s4_model.rotation_model = s4_model.rotation_model.to(device)

# 3.) Create the optimizer and add the parameters we want to optimize
optimizer = optim.LBFGS(
    [s4_model.noise_model.betas_raw, ],
    max_iter=20,
    history_size=5)

# 4.) Create the DataLoader
if batch_size == -1:
    batch_size = x_norm.shape[0]
    # upload the data to the device
    science_norm_flatten = science_norm_flatten.to(device)

data_loader = DataLoader(
    science_norm_flatten,
    batch_size=batch_size,
    shuffle=False)

# 5.) Run the fine-tuning
# needed for gradient accumulation in order to normalize the loss
num_steps_per_epoch = len(data_loader)

for epoch in tqdm(range(num_epochs)):
    for tmp_frames in data_loader:
        # 0.) upload tmp_frames if needed
        if tmp_frames.device != torch.device(device):
            tmp_frames = tmp_frames.to(device)

        def full_closure():
            optimizer.zero_grad()

            # 1.) run the forward path of the noise model
            s4_model.noise_model.compute_betas()
            noise_estimate = s4_model.noise_model(tmp_frames)

            # 2.) compute the residual and rotate it
            residual_sequence = tmp_frames - noise_estimate
            residual_sequence = residual_sequence.view(
                residual_sequence.shape[0],
                1,
                s4_model.data_image_size,
                s4_model.data_image_size)

            rotated_frames = s4_model.rotation_model(
                residual_sequence,
                parang_idx=torch.arange(len(residual_sequence)))

            # 2.) Compute the loss
            frames_no_planet = rotated_frames - torch.mean(rotated_frames, axis=0)
            
            temporal_spectrum = torch.fft.rfft(frames_no_planet, axis=0).abs()**2
            scaling = torch.flip(torch.arange(temporal_spectrum.shape[0]), dims=[0])
            temporal_spectrum *= scaling.to(device).view(-1, 1, 1, 1)
            temporal_spectrum[:5] *= 50
            
            loss_recon = temporal_spectrum.sum()

            loss_reg = (s4_model.noise_model.betas_raw ** 2).sum() \
                * s4_model.noise_model.lambda_reg \
                / num_steps_per_epoch / rotated_frames.shape[0]

            # 3.) Backward
            loss = loss_recon + loss_reg
            loss.backward()

            # get the residual
            residual = torch.mean(rotated_frames, axis=0)[0].detach().cpu().numpy()

            return loss, residual

        def loss_closure():
            return full_closure()[0]


        optimizer.step(loss_closure)
        # 4.) Track the current loss
        current_loss, current_residual = full_closure()

    # 5.) Logg the information
    s4_model._logg_loss_values(
        epoch=epoch,
        loss_recon=current_loss,
        loss_reg=0)

    s4_model.tensorboard_logger.add_image(
        "Images/Residual",
        s4_model._normalize_for_tensorboard(current_residual),
        epoch,
        dataformats="HW")

    if epoch % 20==19:
        tmp_residual_dir = s4_model.residuals_dir / \
            Path(s4_model.fine_tune_start_time)
        tmp_residual_dir.mkdir(exist_ok=True)

        save_as_fits(
            current_residual,
            tmp_residual_dir /
            Path("Residual_epoch_" + str(epoch).zfill(4)
                 + ".fits"),
            overwrite=True)

# 7.) Clean up GPU
s4_model.noise_model = s4_model.noise_model.cpu()
s4_model.rotation_model = s4_model.rotation_model.cpu()
torch.cuda.empty_cache()

  0%|          | 0/500 [00:00<?, ?it/s]

# Test pytorch FFT

In [ ]:
test_signal1 = torch.sin(torch.arange(1000)/20)
test_signal2 = torch.sin(torch.arange(1000)/2)

In [ ]:
plt.plot(test_signal1)
plt.plot(test_signal2)

In [ ]:
plt.plot(torch.fft.fft(test_signal1).abs())
plt.plot(torch.fft.fft(test_signal2).abs())

In [ ]:
torch.flip(torch.arange(10),dims=[0])